In [2]:
!pip install pywin32 pillow

  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl (7.0 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# Windows only
import ctypes
from ctypes import wintypes

gdi32  = ctypes.windll.gdi32

# --- Win32 structs ---
class RECT(ctypes.Structure):
    _fields_ = [("left",   wintypes.LONG),
                ("top",    wintypes.LONG),
                ("right",  wintypes.LONG),
                ("bottom", wintypes.LONG)]

class SIZEL(ctypes.Structure):
    _fields_ = [("cx", wintypes.LONG),
                ("cy", wintypes.LONG)]

class ENHMETAHEADER(ctypes.Structure):
    _fields_ = [
        ("iType",        wintypes.DWORD),
        ("nSize",        wintypes.DWORD),
        ("rclBounds",    RECT),           # pixels (recording DC)
        ("rclFrame",     RECT),           # .01 mm
        ("dSignature",   wintypes.DWORD),
        ("nVersion",     wintypes.DWORD),
        ("nBytes",       wintypes.DWORD),
        ("nRecords",     wintypes.DWORD),
        ("nHandles",     wintypes.WORD),
        ("sReserved",    wintypes.WORD),
        ("nDescription", wintypes.DWORD),
        ("offDescription",wintypes.DWORD),
        ("nPalEntries",  wintypes.DWORD),
        ("szlDevice",    SIZEL),          # pixels of reference device
        ("szlMillimeters",SIZEL),         # mm of reference device
        ("cbPixelFormat",wintypes.DWORD),
        ("offPixelFormat",wintypes.DWORD),
        ("bOpenGL",      wintypes.DWORD),
        ("szlMicrometers",SIZEL)          # not always populated
    ]

# API prototypes
gdi32.GetEnhMetaFileW.argtypes      = [wintypes.LPCWSTR]
gdi32.GetEnhMetaFileW.restype       = wintypes.HANDLE
gdi32.GetEnhMetaFileHeader.argtypes = [wintypes.HANDLE, wintypes.UINT, ctypes.c_void_p]
gdi32.GetEnhMetaFileHeader.restype  = wintypes.UINT
gdi32.DeleteEnhMetaFile.argtypes    = [wintypes.HANDLE]
gdi32.DeleteEnhMetaFile.restype     = wintypes.BOOL

def read_emf_bbox(path, dpi=96.0):
    hemf = gdi32.GetEnhMetaFileW(path)
    if not hemf:
        raise OSError("Failed to open EMF")

    try:
        hdr = ENHMETAHEADER()
        got = gdi32.GetEnhMetaFileHeader(hemf, ctypes.sizeof(hdr), ctypes.byref(hdr))
        if got == 0:
            raise OSError("GetEnhMetaFileHeader failed")

        # Bounds in pixels (recording DC)
        bw = hdr.rclBounds.right - hdr.rclBounds.left
        bh = hdr.rclBounds.bottom - hdr.rclBounds.top

        # Frame in 0.01 mm → mm → inches
        fw_01mm = hdr.rclFrame.right - hdr.rclFrame.left
        fh_01mm = hdr.rclFrame.bottom - hdr.rclFrame.top
        fw_mm   = fw_01mm / 100.0
        fh_mm   = fh_01mm / 100.0
        fw_in   = fw_mm / 25.4
        fh_in   = fh_mm / 25.4

        # Convert to pixels at requested DPI
        fw_px = int(round(fw_in * dpi))
        fh_px = int(round(fh_in * dpi))

        return {
            "bounds_pixels": {"width_px": bw, "height_px": bh},
            "frame_0.01mm":  {"width_0.01mm": fw_01mm, "height_0.01mm": fh_01mm},
            "frame_mm":      {"width_mm": fw_mm, "height_mm": fh_mm},
            "frame_inches":  {"width_in": fw_in, "height_in": fh_in},
            "frame_pixels_at_dpi": {"dpi": dpi, "width_px": fw_px, "height_px": fh_px},
            "device_pixels_hint":  {"cx": hdr.szlDevice.cx, "cy": hdr.szlDevice.cy},
            "device_mm_hint":      {"cx_mm": hdr.szlMillimeters.cx, "cy_mm": hdr.szlMillimeters.cy},
        }
    finally:
        gdi32.DeleteEnhMetaFile(hemf)


In [17]:
import ctypes
from ctypes import wintypes
from PIL import Image
import win32ui

gdi32 = ctypes.windll.gdi32
user32 = ctypes.windll.user32

def emf_to_png(emf_path, png_path, width, height):
    # Load EMF
    hemf = gdi32.GetEnhMetaFileW(emf_path)
    if not hemf:
        raise OSError("Failed to load EMF")

    # Screen DC
    hdc_screen = user32.GetDC(0)
    dc = win32ui.CreateDCFromHandle(hdc_screen)

    # Memory DC + bitmap
    mem_dc = dc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(dc, width, height)
    mem_dc.SelectObject(bmp)

    # White background
    brush = win32ui.CreateBrush()
    mem_dc.FillSolidRect((0, 0, width, height), 0xFFFFFF)

    # Render EMF into memory DC
    rect = wintypes.RECT(0, 0, width, height)
    gdi32.PlayEnhMetaFile(mem_dc.GetSafeHdc(), hemf, ctypes.byref(rect))

    # Save bitmap to file
    bmp.SaveBitmapFile(mem_dc, "temp.bmp")
    Image.open("temp.bmp").save(png_path, "PNG")

    # Cleanup
    gdi32.DeleteEnhMetaFile(hemf)
    mem_dc.DeleteDC()
    dc.DeleteDC()
    user32.ReleaseDC(0, hdc_screen)



In [18]:
INPUT_FILE = "input.emf"
OUTPUT_FILE = "output.png"
DPI = 300

info = read_emf_bbox(INPUT_FILE, dpi=DPI)
width = info['frame_pixels_at_dpi']['width_px']
height = info['frame_pixels_at_dpi']['height_px']
emf_to_png(INPUT_FILE, OUTPUT_FILE, width, height)